# [1141. User Activity for the Past 30 Days I](https://leetcode.com/problems/user-activity-for-the-past-30-days-i/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Activity

<pre>+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| user_id       | int     |
| session_id    | int     |
| activity_date | date    |
| activity_type | enum    |
+---------------+---------+</pre>
This table may have duplicate rows.
The activity_type column is an ENUM (category) of type ('open_session', 'end_session', 'scroll_down', 'send_message').
The table shows the user activities for a social media website. 
Note that each session belongs to exactly one user.
 

Write a solution to find the daily active user count for a period of 30 days ending 2019-07-27 inclusively. A user was active on someday if they made at least one activity on that day.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Activity table:
<pre>+---------+------------+---------------+---------------+
| user_id | session_id | activity_date | activity_type |
+---------+------------+---------------+---------------+
| 1       | 1          | 2019-07-20    | open_session  |
| 1       | 1          | 2019-07-20    | scroll_down   |
| 1       | 1          | 2019-07-20    | end_session   |
| 2       | 4          | 2019-07-20    | open_session  |
| 2       | 4          | 2019-07-21    | send_message  |
| 2       | 4          | 2019-07-21    | end_session   |
| 3       | 2          | 2019-07-21    | open_session  |
| 3       | 2          | 2019-07-21    | send_message  |
| 3       | 2          | 2019-07-21    | end_session   |
| 4       | 3          | 2019-06-25    | open_session  |
| 4       | 3          | 2019-06-25    | end_session   |
+---------+------------+---------------+---------------+</pre>
Output: 
<pre>+------------+--------------+ 
| day        | active_users |
+------------+--------------+ 
| 2019-07-20 | 2            |
| 2019-07-21 | 2            |
+------------+--------------+</pre>
Explanation: Note that we do not care about days with zero active users.

In [1]:
#pandas schema
import pandas as pd

data = [[1, 1, '2019-07-20', 'open_session'], [1, 1, '2019-07-20', 'scroll_down'], [1, 1, '2019-07-20', 'end_session'],
        [2, 4, '2019-07-20', 'open_session'], [2, 4, '2019-07-21', 'send_message'], [2, 4, '2019-07-21', 'end_session'],
        [3, 2, '2019-07-21', 'open_session'], [3, 2, '2019-07-21', 'send_message'], [3, 2, '2019-07-21', 'end_session'],
        [4, 3, '2019-06-25', 'open_session'], [4, 3, '2019-06-25', 'end_session']]
activity = pd.DataFrame(data, columns=['user_id', 'session_id', 'activity_date', 'activity_type']).astype(
    {'user_id': 'Int64', 'session_id': 'Int64', 'activity_date': 'datetime64[ns]', 'activity_type': 'object'})

In [2]:
#converting to spark dataframe
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

activity_df = spark.createDataFrame(activity)
activity_df.show()

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/10 06:25:41 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/10 06:25:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/10 06:25:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------+----------+-------------------+-------------+
|user_id|session_id|      activity_date|activity_type|
+-------+----------+-------------------+-------------+
|      1|         1|2019-07-20 00:00:00| open_session|
|      1|         1|2019-07-20 00:00:00|  scroll_down|
|      1|         1|2019-07-20 00:00:00|  end_session|
|      2|         4|2019-07-20 00:00:00| open_session|
|      2|         4|2019-07-21 00:00:00| send_message|
|      2|         4|2019-07-21 00:00:00|  end_session|
|      3|         2|2019-07-21 00:00:00| open_session|
|      3|         2|2019-07-21 00:00:00| send_message|
|      3|         2|2019-07-21 00:00:00|  end_session|
|      4|         3|2019-06-25 00:00:00| open_session|
|      4|         3|2019-06-25 00:00:00|  end_session|
+-------+----------+-------------------+-------------+


In [3]:
# solving in spark dataframe
import pyspark.sql.functions as F

activity_df.filter(
    (F.col('activity_date') <= '2019-07-27')
    &
    (F.col('activity_date') >= F.date_sub(F.lit('2019-07-27'), 29))) \
    .groupBy('activity_date').agg(F.count_distinct('user_id').alias('active_users')) \
    .show()

+-------------------+------------+
|      activity_date|active_users|
+-------------------+------------+
|2019-07-21 00:00:00|           2|
|2019-07-20 00:00:00|           2|
+-------------------+------------+


In [4]:
# solving in spark SQL

activity_df.createOrReplaceTempView('activity')

spark.sql('''
select activity_date as day, count(distinct user_id) as active_users
from activity
-- for mysql:  where activity_date<='2019-07-27' and activity_date>=date_sub('2019-07-27', interval 29 day)
where activity_date<='2019-07-27' and activity_date>=date_sub('2019-07-27',  29)
group by activity_date;
''').show()

+-------------------+------------+
|                day|active_users|
+-------------------+------------+
|2019-07-21 00:00:00|           2|
|2019-07-20 00:00:00|           2|
+-------------------+------------+


In [5]:
spark.stop()